In [28]:
# pip install alpaca-trade-api

In [30]:
# pip install alpaca-py

In [70]:
# pip install newsapi-python

In [111]:
from newsapi import NewsApiClient
import requests
from bs4 import BeautifulSoup
import pytz
from datetime import datetime
import time

In [ ]:
# Initialize NewsApiClient with your News API key
newsapi = NewsApiClient(api_key='')

In [190]:
# Define the query parameters
query = 'Nvidia'
language = 'en'
sort_by = 'relevancy'
country='us'


In [208]:
# Fetch news articles
news_articles = newsapi.get_everything(q=query, language=language, sort_by=sort_by, from_param='2024-06-30',
                                      to='2024-07-30')

In [209]:
len(news_articles['articles'])

100

In [222]:
article_title = []
url_list = []
date_list = []
content_list = []
for i in range(len(news_articles['articles'])):
    article_title.append(news_articles['articles'][i]['title'])
    url_list.append(news_articles['articles'][i]['url'])
    date_list.append(news_articles['articles'][i]['publishedAt'])

In [227]:
# Create DataFrame
df = pd.DataFrame({
    'Title': article_title,
    'URL': url_list,
    'publishedAt': date_list,
})

In [229]:
df['date'] = pd.to_datetime(df['publishedAt'], format='%Y-%m-%dT%H:%M:%SZ', utc=True)

In [238]:
df = df[df.Title!='[Removed]'].sort_values('date')

In [247]:
df.Title

,Title,URL,publishedAt,date
40,Elon Musk's now lauding Jensen Huang for havin...,https://www.businessinsider.com/elon-musk-prai...,2024-07-01T06:32:09Z,2024-07-01 06:32:09+00:00
44,Zacks Investment Ideas feature highlights: Nvi...,https://consent.yahoo.com/v2/collectConsent?se...,2024-07-01T12:48:00Z,2024-07-01 12:48:00+00:00
79,Exclusive-Nvidia set to face French antitrust ...,https://consent.yahoo.com/v2/collectConsent?se...,2024-07-01T14:49:44Z,2024-07-01 14:49:44+00:00
7,French Antitrust Regulators Preparing Nvidia C...,https://slashdot.org/story/24/07/01/1528252/fr...,2024-07-01T15:28:00Z,2024-07-01 15:28:00+00:00
80,Best gaming laptop 4th of July deals,https://www.digitaltrends.com/computing/gaming...,2024-07-01T16:00:27Z,2024-07-01 16:00:27+00:00
...,...,...,...,...
94,The Morning After: Mark Zuckerberg is surprisi...,https://consent.yahoo.com/v2/collectConsent?se...,2024-07-30T11:57:16Z,2024-07-30 11:57:16+00:00
36,Apple Used Google Tensor Chips to Develop Appl...,https://www.macrumors.com/2024/07/30/google-ch...,2024-07-30T12:56:09Z,2024-07-30 12:56:09+00:00
14,Zuckerberg’s Vision of the Future: A Poorly Re...,https://gizmodo.com/zuckerbergs-vision-of-the-...,2024-07-30T14:40:41Z,2024-07-30 14:40:41+00:00
39,We just learned something surprising about how...,https://www.digitaltrends.com/computing/apple-...,2024-07-30T18:06:54Z,2024-07-30 18:06:54+00:00


In [248]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

/Users/arnabchakraborty/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [249]:
# Initialize the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [250]:
# Encode the titles
embeddings = model.encode(df['Title'].tolist())

In [251]:
# Calculate cosine similarity
similarity_matrix = cosine_similarity(embeddings)

In [272]:
threshold = 0.75  # You can adjust this value

In [273]:
# Function to find groups of similar titles
def find_similar_groups(similarity_matrix, threshold):
    groups = []
    n = similarity_matrix.shape[0]
    used = set()
    
    for i in range(n):
        if i not in used:
            group = [i]
            used.add(i)
            for j in range(i+1, n):
                if j not in used and similarity_matrix[i, j] > threshold:
                    group.append(j)
                    used.add(j)
            groups.append(group)
    
    return groups

In [274]:
# Find groups of similar titles
similar_groups = find_similar_groups(similarity_matrix, threshold)

# Keep only the first title from each group
indices_to_keep = [group[0] for group in similar_groups]

# Create a new dataframe with only the unique titles
df_unique = df.iloc[indices_to_keep].reset_index(drop=True)

In [275]:
df_unique

,Title,URL,publishedAt,date
0,Elon Musk's now lauding Jensen Huang for havin...,https://www.businessinsider.com/elon-musk-prai...,2024-07-01T06:32:09Z,2024-07-01 06:32:09+00:00
1,Zacks Investment Ideas feature highlights: Nvi...,https://consent.yahoo.com/v2/collectConsent?se...,2024-07-01T12:48:00Z,2024-07-01 12:48:00+00:00
2,Exclusive-Nvidia set to face French antitrust ...,https://consent.yahoo.com/v2/collectConsent?se...,2024-07-01T14:49:44Z,2024-07-01 14:49:44+00:00
3,Best gaming laptop 4th of July deals,https://www.digitaltrends.com/computing/gaming...,2024-07-01T16:00:27Z,2024-07-01 16:00:27+00:00
4,Elon Musk gives a hint at just how much his AI...,https://www.businessinsider.com/elon-musk-hint...,2024-07-01T16:18:48Z,2024-07-01 16:18:48+00:00
...,...,...,...,...
80,The Morning After: Mark Zuckerberg is surprisi...,https://consent.yahoo.com/v2/collectConsent?se...,2024-07-30T11:57:16Z,2024-07-30 11:57:16+00:00
81,Apple Used Google Tensor Chips to Develop Appl...,https://www.macrumors.com/2024/07/30/google-ch...,2024-07-30T12:56:09Z,2024-07-30 12:56:09+00:00
82,Zuckerberg’s Vision of the Future: A Poorly Re...,https://gizmodo.com/zuckerbergs-vision-of-the-...,2024-07-30T14:40:41Z,2024-07-30 14:40:41+00:00
83,We just learned something surprising about how...,https://www.digitaltrends.com/computing/apple-...,2024-07-30T18:06:54Z,2024-07-30 18:06:54+00:00


In [276]:
# Print the results
print(f"Original number of titles: {len(df)}")
print(f"Number of unique titles after removing similar ones: {len(df_unique)}")

Original number of titles: 98
Number of unique titles after removing similar ones: 85


In [280]:
df

,Title,URL,publishedAt,date
40,Elon Musk's now lauding Jensen Huang for havin...,https://www.businessinsider.com/elon-musk-prai...,2024-07-01T06:32:09Z,2024-07-01 06:32:09+00:00
44,Zacks Investment Ideas feature highlights: Nvi...,https://consent.yahoo.com/v2/collectConsent?se...,2024-07-01T12:48:00Z,2024-07-01 12:48:00+00:00
79,Exclusive-Nvidia set to face French antitrust ...,https://consent.yahoo.com/v2/collectConsent?se...,2024-07-01T14:49:44Z,2024-07-01 14:49:44+00:00
7,French Antitrust Regulators Preparing Nvidia C...,https://slashdot.org/story/24/07/01/1528252/fr...,2024-07-01T15:28:00Z,2024-07-01 15:28:00+00:00
80,Best gaming laptop 4th of July deals,https://www.digitaltrends.com/computing/gaming...,2024-07-01T16:00:27Z,2024-07-01 16:00:27+00:00
...,...,...,...,...
94,The Morning After: Mark Zuckerberg is surprisi...,https://consent.yahoo.com/v2/collectConsent?se...,2024-07-30T11:57:16Z,2024-07-30 11:57:16+00:00
36,Apple Used Google Tensor Chips to Develop Appl...,https://www.macrumors.com/2024/07/30/google-ch...,2024-07-30T12:56:09Z,2024-07-30 12:56:09+00:00
14,Zuckerberg’s Vision of the Future: A Poorly Re...,https://gizmodo.com/zuckerbergs-vision-of-the-...,2024-07-30T14:40:41Z,2024-07-30 14:40:41+00:00
39,We just learned something surprising about how...,https://www.digitaltrends.com/computing/apple-...,2024-07-30T18:06:54Z,2024-07-30 18:06:54+00:00


In [173]:
article_list = []
news_articles['articles'][0]['publishedAt']

'2024-07-29T22:03:49Z'

In [166]:
news_articles['articles'][0]['title']

'Apple says it uses no Nvidia GPUs to train its AI models'

In [92]:
date_str = news_articles['articles'][0]['publishedAt']

In [91]:
# Define the format string
format_str = "%Y-%m-%dT%H:%M:%SZ"

In [93]:
# Convert to datetime object in UTC
date_time_utc = datetime.strptime(date_str, format_str)

In [94]:
# Convert to datetime object in UTC
date_time_utc = datetime.strptime(date_str, format_str)
date_time_utc = date_time_utc.replace(tzinfo=pytz.utc)

# Convert to Eastern Time
eastern = pytz.timezone('US/Eastern')
date_time_et = date_time_utc.astimezone(eastern)

In [95]:
date_time_et

datetime.datetime(2024, 7, 29, 18, 3, 49, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)

In [116]:
date_llm = date_time_et.strftime("%Y-%m-%d")

In [71]:
url = news_articles['articles'][0]['url']

In [51]:
# Fetch the HTML content
response = requests.get(url)
html_content = response.content

In [53]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

In [67]:
title = soup.title.string

In [57]:
paragraphs = soup.find_all('p')

In [68]:
content=''
for p in paragraphs:
    content += p.get_text()
    content += '\n '

In [72]:
import yfinance as yf
import pandas as pd
import numpy as np
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama

In [97]:
# Create a prompt template
summary_template = """
You are an AI assistant specialized in investment advisory. 
Summarize the news artcile with respect to the NVDA stock and interpret the investment sentiment:

Title: {title}

Content: {content}

Summary:"""

In [98]:
summary_prompt = PromptTemplate(template=prompt_template, input_variables=["title", "content"])

In [99]:
llm = Ollama(
    model="llama3"
)

In [100]:
# Create the chain
chain = LLMChain(llm=llm, prompt=summary_prompt)

In [112]:
start = time.time()
response2 = chain.run(title=title, content=content)
end = time.time()

In [113]:
end-start

5.963320970535278

In [106]:
response

"The article discusses Apple's decision to use Google's cloud infrastructure, specifically its tensor processing unit (TPU) chips, to train two AI models for its forthcoming suite of AI tools and features. This is notable because Nvidia produces the most sought-after AI processors and dominates the market with around 80% market share.\n\nApple used a total of 10,240 TPUv4 and TPUv5p chips to build its AI models, which operate on iPhones and other devices. The company did not disclose any use of Nvidia's chips in the research paper.\n\nThe article also provides some background information on Apple's AI efforts, including its recent developer conference where it unveiled new AI features, such as integrating OpenAI's ChatGPT technology into its software.\n\nAs for the investment sentiment, NVDA (Nvidia) shares are expected to be impacted by this news. Since Nvidia dominates the market with around 80% market share and is not being used by Apple in this instance, investors may view this as 

In [118]:
sector_info='''Nvidia operates in the semiconductor industry, specializing in the design and manufacture of graphics processing units (GPUs) for gaming, professional visualization, data centers, AI, and automotive applications. 
Known for its GeForce brand, Nvidia leads the gaming market while also providing high-performance solutions for AI and machine learning in data centers. 
Additionally, Nvidia's DRIVE platform supports autonomous vehicle technology, and its Jetson platform powers edge computing and IoT devices. 
The company's innovations position it at the forefront of emerging trends in AI, data processing, and autonomous driving, driving significant growth and industry influence.
'''

In [128]:
start_date='2024-01-01'

In [129]:
data = yf.download('NVDA', start=start_date)

[*********************100%%**********************]  1 of 1 completed


In [138]:
date_llm

'2024-07-29'

In [130]:
data.reset_index(inplace=True)

In [132]:
dt_pandas = pd.Timestamp(date_llm)

In [139]:
data[data.Date>dt_pandas]

,Date,Open,High,Low,Close,Adj Close,Volume
144,2024-07-30,111.519997,111.989998,102.540001,103.730003,103.730003,486833300
145,2024-07-31,112.900002,118.339996,110.879997,117.019997,117.019997,457637422


In [142]:
table = data[data.Date<=dt_pandas].tail()

In [144]:
table_string = table.to_string(index=False)

In [196]:
# Create a prompt template
trade_template = """
As an expert trading agent who has sufficient trading experience of trading Nvdia Stock.
Trading Sector information: {sector_info}
Date: {date_llm}
Last 5 day stock info: {table_string}
News summary: {response}
Given the above information can you provide me with an investment decision? Please also consider the momentum of the stock price.
Using the following template:
Decision:(Buy/Sell/Hold)
Reasoing:
Answer:
"""

In [198]:
trade_template = """
As an expert trading agent with extensive experience in trading Nvidia stock, analyze the following information:

Sector: {sector_info}
Date: {date_llm}
Recent Stock Performance:
{table_string}

Recent News Summary:
{response}

Based on the provided data, please offer an investment decision. Consider factors such as:
1. Current market trends in the technology sector
2. Nvidia's recent stock price momentum
3. Potential impact of the news on stock performance

Provide your analysis using the following format:

Decision: (Buy/Sell/Hold)
Reasoning: (Explain your decision in 3-5 concise points)
Confidence Level: (Low/Medium/High)

Answer:
"""

In [199]:
trade_prompt = PromptTemplate(template=trade_template, input_variables=["sector_info", "date_llm", "table_string", 
                                                                       "response"])

In [200]:
# Create the chain
chain = LLMChain(llm=llm, prompt=trade_prompt)

In [201]:
start = time.time()
trade = chain.run(sector_info=sector_info, date_llm=date_llm, table_string=table_string, response=response)
end = time.time()

In [202]:
end-start

12.785616874694824

In [203]:
print(trade)

Decision: Sell
Reasoning:

1. Nvidia's recent stock price momentum is downward, with the stock closing lower on July 24th and 25th, indicating a loss of investor confidence.
2. The news about Apple using Google's TPU chips instead of Nvidia's AI processors could be perceived as negative for Nvidia's market share and future growth prospects, potentially leading to further downward pressure on the stock price.
3. The technology sector is currently experiencing significant disruptions and changes, with many companies shifting their focus towards cloud-based solutions and artificial intelligence (AI). This trend may continue to benefit Google and other companies that are well-positioned in these areas.

Confidence Level: Medium

I have a medium level of confidence in this decision due to the uncertainty surrounding the impact of Apple's decision on Nvidia's market share. However, based on the recent stock price trends and the potential negative implications of this news, I believe it is pr

In [154]:
print(trade)

As an expert trading agent with sufficient trading experience of trading Nvidia (NVDA) stock, I would recommend selling NVDA shares based on the current market situation.

Reasons:

1. **Industry influence and growth**: Nvidia is a leading player in the semiconductor industry, specializing in GPUs for gaming, AI, data centers, and automotive applications. The company's innovations position it at the forefront of emerging trends in AI, data processing, and autonomous driving, driving significant growth and industry influence.
2. **Last 5-day stock info**: The recent stock price movement indicates a downward trend, with prices declining by around 6-7% over the past five trading days. This could be a sign that investors are becoming increasingly cautious about Nvidia's future prospects.
3. **News summary**: Apple's decision to use Google's cloud infrastructure for AI model training is a negative development for NVDA, as it reduces the likelihood of Apple using Nvidia's chips in this insta

In [156]:
data['daily_trade_return'] = data['Adj Close'] - data['Open']

In [157]:
data[data.Date>dt_pandas]

,Date,Open,High,Low,Close,Adj Close,Volume,daily_trade_return
144,2024-07-30,111.519997,111.989998,102.540001,103.730003,103.730003,486833300,-7.789993
145,2024-07-31,112.900002,118.339996,110.879997,117.019997,117.019997,457637422,4.119995


In [14]:
# Calculate daily returns
data['Return'] = data['Adj Close'].pct_change()

In [24]:
# Calculate log returns
data['Log Return'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))

In [15]:
# Calculate variance of the returns
variance = data['Return'].var()

In [25]:
mean_return = data['Return'].mean()
mean_log_return = data['Log Return'].mean()
std_dev = data['Return'].std()

In [26]:
mean_log_return

0.005875982800714965

In [19]:
mean_return

0.0064310391363098605

In [20]:
std_dev

0.03315865600421478

In [3]:
# Create a prompt template
prompt_template = """
You are an AI assistant specialized in dietary guidelines. 
Use the following pieces of context to answer the question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Answer:"""

In [4]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

llm = Ollama(
    model="llama3"
)

In [5]:
llm = Ollama(
    model="llama3"
)

In [6]:
# Create the chain
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
def answer_question(query):
    # Search for relevant context
    search_results = search_faiss(query)
    
    # Combine the content from the search results
    context = "\n\n".join([result['content'] for result in search_results])

    # Run the chain
    response = chain.run(context=context, question=query)
    
    return response